<a href="https://colab.research.google.com/github/isaacpy-cloud/TraderWolfs/blob/main/Trading_Selection-First.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-binance
!pip install catboost
!pip install mpl_finance
!pip install tradingview-ta
!pip install websocket-client
!pip install finta
!pip install pandas_ta

In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 161001 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [3]:
!pip install ccxt

In [4]:
import talib as ta
import pandas as pd
import numpy as np
import time
import dateparser
import pytz
import json
import datetime as dt
from datetime import datetime, timedelta
from threading import Thread
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ochl
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool, cv
from tqdm import tqdm as tqdm
import websocket
from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
from time import sleep
from tradingview_ta import TA_Handler, Interval, Exchange
from tradingview_ta import Compute
from requests import get
import socket
import sys
import math
from finta import TA

Api Key, Api Secret Alanı

In [5]:
api_key = 'hXjL8ujRBXbXLNaek31Z4ysCiCmfhnbRiVdApFFt8YlwBST6hUx1gpoVC4r5xjpS'
api_secret = 'zFyTQoSDZVdRvmmFnHgjP4oaPWcXC4hhBbICaavRKY4pBDKJawWOPX7hCF9uSAMw'

Ana Çalışma Fonksiyonum

In [6]:
from binance.client import Client

class BinanceConnection:
  def __init__(self,apikey,apisecret):
    self.apikey = apikey
    self.apisecret = apisecret
    self.ClientCon = Client(self.apikey,self.apisecret)

class HistoricalKlines:
  def ImportKlines(self,Client,Symbol,Interval,Limit):
    try:
      self.Klines = Client.get_klines(symbol=Symbol, interval=Interval, limit=Limit)
      return self.Klines
    except Exception as exp:
      print(exp.status_code, flush=True)
      print(exp.message, flush=True) 

In [7]:
class Indicator_Settings:
  def T3Tilson(self,close_array, high_array, low_array, volume_factor, t3Length):
    ema_first_input = (high_array + low_array + 2 * close_array) / 4
    e1 = ta.EMA(ema_first_input, t3Length)
    e2 = ta.EMA(e1, t3Length)
    e3 = ta.EMA(e2, t3Length)
    e4 = ta.EMA(e3, t3Length)
    e5 = ta.EMA(e4, t3Length)
    e6 = ta.EMA(e5, t3Length)
    c1 = -1 * volume_factor * volume_factor * volume_factor
    c2 = 3 * volume_factor * volume_factor + 3 * volume_factor * volume_factor * volume_factor
    c3 = -6 * volume_factor * volume_factor - 3 * volume_factor - 3 * volume_factor * volume_factor * volume_factor
    c4 = 1 + 3 * volume_factor + volume_factor * volume_factor * volume_factor + 3 * volume_factor * volume_factor
    T3 = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3
    return T3

Class'larımın İçeri Aktarılması ve Bağlantının Sağlanması

In [8]:
BinanceCon = BinanceConnection(api_key,api_secret)
Clients = BinanceCon.ClientCon
Symbol = 'TROYBNB'

Bazı Farklı Sembollerin İçe Aktarılması

In [9]:
# Bu edit sayfasıdır.

In [10]:
KlinesCon = HistoricalKlines()
Indicators = Indicator_Settings()

In [19]:
while True:
  sleep(10)
  # Datamın Alınması
  Historic_Klines = KlinesCon.ImportKlines(Clients,Symbol,Clients.KLINE_INTERVAL_1DAY,500)


  # Datanın İçerisinden Bilgilerin Alınması
  open_time = [int(entry[0]) for entry in Historic_Klines]
  open = [float(entry[1]) for entry in Historic_Klines]
  high = [float(entry[2]) for entry in Historic_Klines]
  low = [float(entry[3]) for entry in Historic_Klines]
  close = [float(entry[4]) for entry in Historic_Klines]

  #Son Değerlerin Alınması
  last_closing_price = close[-1]
  previous_closing_price = close[-2]
  close_array = np.asarray(close)
  close_finished = close_array[:-1]
  high_array = np.asarray(high)
  low_array = np.asarray(low)

  new_time = [datetime.fromtimestamp(time / 100) for time in open_time]
  new_time_x = [date.strftime("%y-%m-%d") for date in new_time]
  volume_factor = 0.7
  t3lenght = 8
  TilsonT3 = Indicators.T3Tilson(close_array,high_array,low_array,volume_factor=volume_factor, t3Length=t3lenght)
  t3_last = TilsonT3[-1]
  t3_previous = TilsonT3[-2]
  t3_prev_previous = TilsonT3[-3]

  # kırmızıdan yeşile dönüyor
  if t3_last > t3_previous and t3_previous < t3_prev_previous:
      print('tillson t3 buy signal, from red to green')

  # yeşilden kırmızıya dönüyor
  elif t3_last < t3_previous and t3_previous > t3_prev_previous:
      print('tillson t3 sell signal, from green to red')   

  #Macd'nin ve RSI'ın Alınması
  macd, macdsignal, macdhist = ta.MACD(close_finished, fastperiod=12, slowperiod=26, signalperiod=9)
  rsi = ta.RSI(close_finished, timeperiod=14)
  if len(macd) > 0:
    last_macd = macd[-1]
    last_macd_signal = macdsignal[-1]
    previous_macd = macd[-2]
    previous_macd_signal = macdsignal[-2]
    rsi_last = rsi[-1]
    macd_cross_up = last_macd > last_macd_signal and previous_macd < previous_macd_signal

    print(rsi_last)
    if macd_cross_up and rsi_last > 50:
      print('al sinyali', flush=True) 

51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918
51.79460411428918


KeyboardInterrupt: ignored